In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json

In [4]:
QDRANT_URL = "http://127.0.0.1:6333"
COLLECTION_NAME = "hybrid_docs"
DATA_PATH = "../data/processed/proc_docx.jsonl"
EMBED_MODEL_NAME = 'intfloat/multilingual-e5-large'
EMBED_DIM = 1024

In [5]:
docs = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        text = data.get("text", "")
        meta = {
            "doc_id": data.get("doc_id", ""),
            "file_path": data.get("file_path", "")
        }
        if text.strip():
            docs.append({"text": text, "metadata": meta})

In [6]:
len(docs)

31

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts, metadatas = [], []
for doc in docs:
    chunks = splitter.split_text(doc["text"])
    texts.extend(chunks)
    metadatas.extend([doc["metadata"]] * len(chunks))

In [8]:
len(texts), len(metadatas)

(53, 53)

In [9]:
texts = ['passage: ' + t for t in texts]

In [10]:
# emb = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
embedder = SentenceTransformer(EMBED_MODEL_NAME)

In [11]:
qdrant = QdrantClient(url=QDRANT_URL)

In [13]:
qdrant.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

dense_vecs = embedder.encode(texts, show_progress_bar=False).tolist()

points = [
    PointStruct(id=i, vector=dense_vecs[i], payload={"text": texts[i]})
    for i in range(len(texts))
]
qdrant.upsert(collection_name=COLLECTION_NAME, points=points)

bm25 = BM25Okapi([t.split() for t in texts])

/tmp/ipykernel_15073/319044017.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


In [14]:
def hybrid_search(query, alpha=0.5, top_k=3):
    """alpha=0 → только BM25, alpha=1 → только dense"""
    dense_query = embedder.encode(query).tolist()
    dense_results = qdrant.search(
        collection_name=COLLECTION_NAME,
        query_vector=dense_query,
        limit=top_k * 3,
    )

    # BM25 результаты
    sparse_scores = bm25.get_scores(query.split())

    # Комбинируем
    combined = []
    for res in dense_results:
        doc_id = res.id
        dense_score = 1 - res.score  # cosine distance → similarity
        sparse_score = sparse_scores[doc_id]
        final_score = alpha * dense_score + (1 - alpha) * sparse_score
        combined.append((final_score, res.payload["text"]))

    combined.sort(reverse=True, key=lambda x: x[0])
    return combined[:top_k]


In [27]:
# query = "Итоги запусков кампаний по категории бытовой химии"
# query = "Успешные кампании кофе и чай"
query = "Итоги запусков кампаний по категории молочная продукция"
query = 'query: ' + query
for score, text in hybrid_search(query, alpha=0.8, top_k=5):
    print(f"{score:.3f} — {text}")
    print('-'*80)


0.419 — passage: [Title] Отчет: Акция — Кондитерка, ПФО, апрель 2025.
 Настоящий отчет подготовлен по результатам акция в регионе ПФО. Основной целью являлось повышение продаж и вовлеченности по категории Кондитерка. Несмотря на корректную реализацию кампании, существенного роста ключевых показателей не зафиксировано. [Title] Методология.
 В анализ были включены данные по транзакциям, CRM и digital-показам. Рассчитаны метрики ROMI, прирост выручки, вовлеченность и коэффициент отклика. [Title] Результаты.
 Uplift составил -0.7%, ROMI — 81%. Значимого отличия между тестовой и контрольной группами не выявлено. [Title] Заключение.
--------------------------------------------------------------------------------
0.385 — passage: [Title] Отчет: Акция — Бакалея, ПФО, февраль 2025.
 Настоящий отчет подготовлен по результатам акция в регионе ПФО. Основной целью являлось повышение продаж и вовлеченности по категории Бакалея. Несмотря на корректную реализацию кампании, существенного роста ключевых

/tmp/ipykernel_15073/1114625531.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  dense_results = qdrant.search(
